# Вступление

Приветсвую тех, кто заглнянул сюда, в мой ноутбук!
Как вы наверное поняли из названия моего репозитория/датасета с Kaggle это ноутбук, посвященный анализу стендов из аниме "Невероятные приключения ДжоДжо"

Сначала немного контекста:

Стенд -- это физическое проявление "жизненной энергии" человека, его сильный дух

У стендов есть характеристики:

- **Разрушительная сила**: Измеряет силу стенда и его способность вызвать разрушения (физические повреждения или сопутствующий ущерб окружающей среде) за определенный период времени.

- **Скорость**: Измеряет ловкость и рефлексы стенда, а также скорость выполнения.

- **Радиус**: Измеряет компромисс диапазона проявления стенда, диапазон влияния способностей и пространственную мобильность.

- **Выносливость**: Измеряет продолжительность времени, в течение которого стенд может активно поддерживать свою способность. Стенды дальней дистанции, способные работать удаленно от своих пользователей, такие как Lovers и Empress, описываются как обладающие высокой выносливостью. Это также относится к материализованным стендам, такие как Yellow Temperance и Atom Heart Father, которые могут привязываться к объектам на длительное время.

- **Точность**: Измеряет точность стенда и радиус воздействия/эффекта его способностей на указанные цели. Стенды автоматического типа обычно оцениваются по рангу D или ниже, за некоторыми исключениями.

- **Потенциал развития**: Измеряет возможные функции стенда, использование его способностей и сил, а также способность улучшить его общие возможности. Ранг уменьшается по мере того, как пользователь осваивает свой стенд.

Пример описания характеристики стенда:

<div style="text-align:center"> 

<img src="https://pm1.aminoapps.com/6955/8540d06f69c1111b7bae390fb9b66effe3809d46r1-1080-1017v2_hq.jpg" alt="drawing" width="200"/>

</div>

Зная это, приступим к анализу данного датасета и найдем что-нибудь интересное!

<div style="text-align:center"> 

![Ой, Джоске](https://avatars.dzeninfra.ru/get-zen_doc/3680683/pub_62dfb5f901cd0904a73542c1_62dfbe04c58eaf42ac4bd2fe/orig)


# 1 часть -- Импорт библиотек

In [275]:
import pandas as pd
import numpy as np

import scipy.stats as sts
import matplotlib.pyplot as plt
import plotly.express as px
 

from IPython.display import Image, display
 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2 часть -- Начало анализа, первый взгялд на данные

In [276]:
begin_dataset = pd.read_csv('JoJo_Bizarre_Adventure_All_Stands_Stats.csv')

# Выведем 10 случайных строк, чтобы посмотреть на датасет
display(begin_dataset.sample(10))

display(begin_dataset.info())


,Stand,PWR,SPD,RNG,STA,PRC,DEV,Story
88,Talking Head,E,E,B,A,E,E,Part 5: Vento Aureo
107,Jumpin' Jack Flash,B,C,B,A,D,E,Part 6: Stone Ocean
10,Osiris,E,D,D,C,D,D,Part 3: Stardust Crusaders
51,Ratt,B,C,D,B,E,C,Part 4: Diamond is Unbreakable
45,Highway Star,C,B,A,A,E,C,Part 4: Diamond is Unbreakable
30,Tower of Gray,E,A,A,C,E,E,Part 3: Stardust Crusaders
55,The Lock,E,E,A,A,E,E,Part 4: Diamond is Unbreakable
126,In a Silent Way,C,C,D,A,D,B,Part 7: Steel Ball Run
39,Cinderella,D,C,C,C,A,C,Part 4: Diamond is Unbreakable
2,Bastet,E,E,B,A,E,E,Part 3: Stardust Crusaders


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Stand   156 non-null    object
 1   PWR     156 non-null    object
 2   SPD     156 non-null    object
 3   RNG     156 non-null    object
 4   STA     156 non-null    object
 5   PRC     156 non-null    object
 6   DEV     156 non-null    object
 7   Story   156 non-null    object
dtypes: object(8)
memory usage: 9.9+ KB


None

# 2.1 -- Части ДжоДжо

Сначала я хочу переимновать колонки статистики для удобства, а также очень хочу посмотреть, сколько у нас есть частей всего (сколько уникальных значений в колонке Story)

In [277]:
# Переименуем колнки
dict_to_replace = {'PWR': 'Сила', 
                   'SPD': 'Скорость',
                   'RNG': 'Дальность',
                   'STA': 'Выносливость',
                   'PRC': 'Точность',
                   'DEV': 'Потенциал'}

begin_dataset.rename(columns=dict_to_replace, inplace=True)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
0,Anubis,B,B,E,A,E,C,Part 3: Stardust Crusaders
1,Atum,D,C,D,B,D,D,Part 3: Stardust Crusaders
2,Bastet,E,E,B,A,E,E,Part 3: Stardust Crusaders
3,Cream,B,B,D,C,C,D,Part 3: Stardust Crusaders
4,Death Thirteen,C,C,E,B,D,B,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,E,A,A,A,E,E,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,A,B,E,E,C,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,E,B,A,A,E,E,Light Novel: Purple Haze Feedback
154,Remote Romance,E,D,infinite,B,C,E,Featured


Теперь посмотрим сначала на то, какие у нас есть части данного аниме, а также посмотрим на распределение характеристик, чтобы понять общую картину

In [278]:
parts_of_jojo = begin_dataset.value_counts('Story')
print(parts_of_jojo)

Story
Part 3: Stardust Crusaders           33
Part 5: Vento Aureo                  29
Part 4: Diamond is Unbreakable       28
Part 6: Stone Ocean                  26
Part 7: Steel Ball Run               24
Light Novel: Purple Haze Feedback     7
Part 8: JoJolion                      7
Featured                              2
Name: count, dtype: int64


Что ж, по крайней мере, я дошел до Stone Ocean, так что я думаю, что большинство стендов мне будет знакомо :)

Большинство стендов находятся в 3 части, это круто, так как нас только знакомили с ними в этом сезоне

Меньше всего стендов находятся в "Featured", честно, сам не знаю, что это такое, так что в теории можно вывести эти стенды и посмотреть на них получше, может я что-нибудь да вспомню

## 2.2 -- Стенды из главы "Featured"

In [279]:
featured_part = begin_dataset[begin_dataset['Story'] == 'Featured']

display(featured_part)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
154,Remote Romance,E,D,infinite,B,C,E,Featured
155,House of Holy,E,B,C,A,D,D,Featured


Я узнал про эти стенды подробнее и оказывается, что **Remote Romance** -- это стенд, разработанный Хирохико Араки (создатель ДжоДжо) для выставки JoJo 2012. Его целью было продвижение художественной выставки через Интернет в прямом эфире.


<div style="text-align:center"> 

<img src="https://static.wikia.nocookie.net/jjba/images/3/32/Remote_Romance.png/revision/latest?cb=20170309021209" alt="drawing" width="200"/>

</div>

Подробнее про **Remote Romance** можете почитать [здесь](https://jojo.fandom.com/ru/wiki/Remote_Romance)

Так что этого стенда и не было в аниме/манге


Теперь узнаем про **House of Holy**. С ним очень интересно все -- это стенд-отель-лабиринт и у него нет внешнего вида, его опасность заключается в том, что любой, кто войдет в отель -- сразу окажется в лабиринте и в течение часа надо найти выход, если человек не найдет выход, то он пропадет в лабиринте навсегда. Любой физический урон неэффективен. 

[Подробнее про сам стенд](https://jojo.fandom.com/ru/wiki/House_of_Holy)

Также он фигурировал в специальной квест-игре, где участники в Японии должны решить загадки владельца стенда и выбраться из лабиринта

[Подробнее про квест-игру](https://jojo.fandom.com/ru/wiki/JoJo%27s_Bizarre_Escape:_The_Hotel)

Можно предположить, что этот стенд появляется в 3 части и его владелец является приспешником Дио (о нем позже поговорим)

## 2.3 -- Общее распределение характеристик

### 2.3.1 -- Сила

Теперь рассмотрим общее распределение характреристик стендов и поймем, стендов каких типов по качеству характеристик (A-E) больше всего

In [280]:
power_stats = begin_dataset['Сила'].value_counts()

display(power_stats)

Сила
E            35
A            32
C            27
B            26
D            17
undefined    16
unknown       2
infinite      1
Name: count, dtype: int64

Давайте, чтобы не путаться, мы будем обозначать характеристики цифрами (от 1 до 5), соответственно, это у нас будет (E-A)

In [281]:
replace_stat_dict = {'A': 5,
                     'B': 4,
                     'C': 3,
                     'D': 2,
                     'E': 1}

dataset_columns = begin_dataset.columns[1:-1]

for i in dataset_columns:
    begin_dataset[i].replace(replace_stat_dict, inplace = True)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback
154,Remote Romance,1,2,infinite,4,3,1,Featured


Посмотрим еще раз

In [282]:
power_stats = begin_dataset['Сила'].value_counts()

display(power_stats)

Сила
1            35
5            32
3            27
4            26
2            17
undefined    16
unknown       2
infinite      1
Name: count, dtype: int64

Как видно, у нас большинство стендов -- слабые по силе *(1)*, но также много и очень сильных *(5)*


Также у нас есть такие параметры, как неопределено, неизвестно и бесконечно

**Бесконечно** понятно, бесконечная мощь, непобедимый по силе, у нас такой только 1 стенд, давайте посмотрим, кто это может быть


In [283]:
inf_power_stand = begin_dataset[begin_dataset['Сила'] == 'infinite']

display(inf_power_stand)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo


А, ну тогда понятно, да, действительно, у нас только один **Gold Experience Requiem** (GER) настолько сильный :)

Бесконечное все -- непобедимый стенд, действительно

<p align="center">
    <img src="https://steamuserimages-a.akamaihd.net/ugc/775103339591880851/0452250037714568022DC0681A3AD2704C721D54/?imw=5000&amp;imh=5000&amp;ima=fit&amp;impolicy=Letterbox&amp;imcolor=%23000000&amp;letterbox=false" alt="Gold Experience Requiem 1" width="300">
</p>

<p align="center">
    <img src="https://i.pinimg.com/originals/6a/cc/c6/6accc6d646fb378c932fcd2b73e16307.png" alt="Gold Experience Requiem 2" width="300">
</p>

<p align="center">
    <img src="https://steamuserimages-a.akamaihd.net/ugc/5108802431775556322/46F42EC084464AA428E7703388BA664F0A560A9C/?imw=512&amp;imh=287&amp;ima=fit&amp;impolicy=Letterbox&amp;imcolor=%23000000&amp;letterbox=true" alt="Gold Experience Requiem 3" width="300">
</p>


Рассмотрим теперь стенды с пометкой *undefined* и *unknown* и решим, они все таки сильные или слабые

In [284]:
unknown_stands = begin_dataset[
    begin_dataset['Сила'] == 'unknown'
]

undefined_stands = begin_dataset[
    begin_dataset['Сила'] == 'undefined'
]

display(unknown_stands)
display(undefined_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
90,Whitesnake,unknown,2,unknown,5,unknown,unknown,Part 6: Stone Ocean
104,"Green, Green Grass of Home",unknown,unknown,unknown,unknown,unknown,unknown,Part 6: Stone Ocean


,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
85,Rolling Stones,undefined,4,5,5,1,undefined,Part 5: Vento Aureo
91,C-Moon,undefined,4,4,unknown,unknown,unknown,Part 6: Stone Ocean
92,Jail House Lock,undefined,3,4,5,undefined,undefined,Part 6: Stone Ocean
95,Under World,undefined,3,5,3,undefined,undefined,Part 6: Stone Ocean
96,Sky High,undefined,undefined,4,3,undefined,undefined,Part 6: Stone Ocean
99,Bohemian Rhapsody,undefined,undefined,infinite,5,undefined,undefined,Part 6: Stone Ocean
100,Burning Down the House,undefined,undefined,undefined,undefined,undefined,undefined,Part 6: Stone Ocean
102,Dragon's Dream,undefined,undefined,undefined,5,undefined,undefined,Part 6: Stone Ocean
108,Limp Bizkit,undefined,4,4,5,3,1,Part 6: Stone Ocean
116,20th Century Boy,undefined,3,undefined,5,2,3,Part 7: Steel Ball Run


Ну... В теории можно построить график распределения и +- случайно распределить параметры, на основе существующих данных. Допустим, мы присвоим 1 с такой вероятностью, 2 с другой и т.д

Алгоритм примерно следующий:

находим сколько стендов имеют силу 1, например, и делим на общее количество стендов

In [285]:
done_values_power = begin_dataset[
    (begin_dataset['Сила'] != 'undefined') & (begin_dataset['Сила'] != 'unknown') & (begin_dataset['Сила'] != 'infinite')
]

done_values_power_piv = done_values_power.value_counts('Сила').reset_index()
print(done_values_power_piv)

power_stats_histogram = px.histogram(done_values_power_piv, x = 'Сила', y = 'count', nbins = 8, color_discrete_sequence=['#f40b12'])
power_stats_histogram.update_layout(title = {'text': 'Распределение силы', 'x': 0.5}, showlegend = False, xaxis_title = 'Сила', yaxis_title = 'Количество')
power_stats_histogram.show()




   Сила  count
0     1     35
1     5     32
2     3     27
3     4     26
4     2     17


In [286]:
additional_done_values_power = done_values_power.value_counts('Сила').reset_index().rename(columns = {'count': 'Кол-во'})

additional_done_values_power['probability'] = additional_done_values_power['Кол-во'] / additional_done_values_power['Кол-во'].sum()

display(additional_done_values_power)

,Сила,Кол-во,probability
0,1,35,0.255474
1,5,32,0.233577
2,3,27,0.197080
3,4,26,0.189781
4,2,17,0.124088


Теперь, имея данные вероятности, мы можем заполнить наши значения

In [287]:
stands_to_be_replaced = begin_dataset[
    (begin_dataset['Сила'] == 'unknown') | (begin_dataset['Сила'] == 'undefined')
]

def replace_empty_power(row):
    row['Сила'] = np.random.choice(additional_done_values_power['Сила'], p = additional_done_values_power['probability'])
    return row['Сила']

stands_to_be_replaced['Сила'] = stands_to_be_replaced.apply(replace_empty_power, axis = 1)

begin_dataset.loc[stands_to_be_replaced.index, 'Сила'] = stands_to_be_replaced['Сила']

display(begin_dataset)

display(begin_dataset.value_counts('Сила').reset_index())


C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\1309006916.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback
154,Remote Romance,1,2,infinite,4,3,1,Featured


,Сила,count
0,1,39
1,5,36
2,3,30
3,4,30
4,2,20
5,infinite,1


Предположим, что стенды с силой 3 и больше, а также таинственный стенд, у которого сила бесконечна -- сильные, остальные -- слабые, тогда давайте отобразим их распределение на круговой диаграмме (как раз будут хорошие категории)

In [288]:
def assign_strongest_stands(row):
    if row['Сила'] == 'infinite' or row['Сила'] >= 3:
        row['strong'] = 'Сильный стенд'
    else:
        row['strong'] = 'Слабый стенд'
    return row['strong']

begin_dataset['strong'] = begin_dataset.apply(assign_strongest_stands, axis = 1)



In [289]:
pivot_strong_stands = begin_dataset.value_counts('strong').reset_index().rename(columns = {'count': 'Количество'})

display(pivot_strong_stands)

strong_stands_pie = px.pie(pivot_strong_stands, names='strong', values='Количество', width = 500, height = 500, hole = 0.5, color_discrete_sequence=['#db2724', '#24d8db'])
strong_stands_pie.update_layout(title = {'text': 'Распределение стендов по силе', 'x': 0.5}, showlegend = False)
strong_stands_pie.show()




,strong,Количество
0,Сильный стенд,97
1,Слабый стенд,59


Все равно по распределению видно, что у нас **больше именно сильных стендов**

### 2.3.2 -- Скорость

Теперь посмотрим, что там по скорости стендов

In [290]:
speed_stats = begin_dataset['Скорость'].value_counts()

display(speed_stats)

Скорость
4            41
3            35
5            29
1            27
2            14
undefined     6
infinite      3
unknown       1
Name: count, dtype: int64

Как видно, у нас почти очень быстрые стенды получились, причем медленных прямо мало

Посмотрим на стенды underfined и unknown

In [291]:
display(begin_dataset[(begin_dataset['Скорость'] == 'undefined') | (begin_dataset['Скорость'] == 'unknown')])

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
96,Sky High,4,undefined,4,3,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
99,Bohemian Rhapsody,1,undefined,infinite,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд
100,Burning Down the House,2,undefined,undefined,undefined,undefined,undefined,Part 6: Stone Ocean,Слабый стенд
102,Dragon's Dream,3,undefined,undefined,5,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
104,"Green, Green Grass of Home",5,unknown,unknown,unknown,unknown,unknown,Part 6: Stone Ocean,Сильный стенд
141,California King Bed,4,undefined,1,4,1,1,Part 8: JoJolion,Сильный стенд
144,Paisley Park,3,undefined,5,5,2,3,Part 8: JoJolion,Сильный стенд


Поступим с такими значениями скорости точно также, как и с силой, просто распределим остальные параеметры на основе имеющихся

In [292]:
stands_speed_stats = begin_dataset[
    (begin_dataset['Скорость'] != 'undefined') & (begin_dataset['Скорость'] != 'unknown') & (begin_dataset['Скорость'] != 'infinite')
]

display(stands_speed_stats)



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд
...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд


Построим график распределения

In [293]:
pivot_speed_stands = stands_speed_stats.value_counts('Скорость').reset_index()

display(pivot_speed_stands)

stands_speed_histogram = px.histogram(pivot_speed_stands, x = 'Скорость', y = 'count', nbins = 8, color_discrete_sequence=['#36e619'])

stands_speed_histogram.show()

,Скорость,count
0,4,41
1,3,35
2,5,29
3,1,27
4,2,14


In [294]:
sum_speed = pivot_speed_stands['count'].sum()

pivot_speed_stands['probability'] = pivot_speed_stands['count'] / sum_speed

display(pivot_speed_stands)

,Скорость,count,probability
0,4,41,0.280822
1,3,35,0.239726
2,5,29,0.198630
3,1,27,0.184932
4,2,14,0.095890


In [295]:
stands_speed_to_replace = begin_dataset[(begin_dataset['Скорость'] == 'undefined') | (begin_dataset['Скорость'] == 'unknown')]

display(stands_speed_to_replace)

def replace_speed(row):
    if row['Скорость'] == 'undefined' or row['Скорость'] == 'unknown':
        row['Скорость'] = np.random.choice(pivot_speed_stands['Скорость'], p = pivot_speed_stands['probability'])
    return row['Скорость']

stands_speed_to_replace['Скорость'] = stands_speed_to_replace.apply(replace_speed, axis = 1)

begin_dataset.loc[stands_speed_to_replace.index, 'Скорость'] = stands_speed_to_replace['Скорость']

display(begin_dataset)

display(begin_dataset.value_counts('Скорость').reset_index())

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
96,Sky High,4,undefined,4,3,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
99,Bohemian Rhapsody,1,undefined,infinite,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд
100,Burning Down the House,2,undefined,undefined,undefined,undefined,undefined,Part 6: Stone Ocean,Слабый стенд
102,Dragon's Dream,3,undefined,undefined,5,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
104,"Green, Green Grass of Home",5,unknown,unknown,unknown,unknown,unknown,Part 6: Stone Ocean,Сильный стенд
141,California King Bed,4,undefined,1,4,1,1,Part 8: JoJolion,Сильный стенд
144,Paisley Park,3,undefined,5,5,2,3,Part 8: JoJolion,Сильный стенд


C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\2724405903.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд
...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд


,Скорость,count
0,4,43
1,3,38
2,5,29
3,1,28
4,2,15
5,infinite,3


У нас появились 3 стенда с бесконечной скоростью, интересно, посмотрим на них

In [296]:
infinite_speed_stands = begin_dataset[
    begin_dataset['Скорость'] == 'infinite'
]

display(infinite_speed_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд
83,Notorious B.I.G,5,infinite,infinite,infinite,1,5,Part 5: Vento Aureo,Сильный стенд
109,Made in Heaven,4,infinite,3,5,3,5,Part 6: Stone Ocean,Сильный стенд


Тут у нас уже знакомый GER (Gold Experience Requiem)

Также у нас здесь есть **Notorius B.I.G**, да, это поистине устрашающий стенд, несмотря на то, что его обладатель умер, стенд продолжает преследовать все вокруг, подстраивается под пространство и поглощает абсолютно все. Скорость зависит от того, кого он преслудует. будет преследовать улитку -- будет двигаться со скоростью чуть большей улитики, будет преследовать самолет -- значит будет двигаться быстрее самолета

<p align="center">
    <img src="https://i.ytimg.com/vi/qvk0zrIqm4w/maxresdefault.jpg" alt="Да... Страшный стенд" height = "200", width="400">
</p>

Про **Made in Heaven**, к сожалению, я не знаю ничего, так как в самом разгаре просмотра Stone Ocean, но все равно приложу фотку стенда, ~~будет мне спойлером :)~~

<p align="center">
    <img src="https://static.wikia.nocookie.net/zlodei/images/b/bb/Made_in_Heaven.webp/revision/latest/scale-to-width-down/562?cb=20240217153358&path-prefix=ru" alt="Да... Страшный стенд" height = "400", width="200">
</p>

Да, неплохая лошадка, выглядит мощной, раз я там видел Пуччи, то думаю, это его реквием, может быть 🤔


Теперь построим круговую диаграмму, аналогичную с силой и посмотрим, каких стендов больше -- быстрых или медленных. Логика та же -- стенды, у которых скорость >= 3 -- быстрые, меньше -- медленные

In [297]:
def assign_speed_stands(row):
    if row['Скорость'] == 'infinite' or row['Скорость'] >= 3:
        row['speed'] = 'Быстрый стенд'
    else:
        row['speed'] = 'Медленный стенд'
    return row['speed']

begin_dataset['speed'] = begin_dataset.apply(assign_speed_stands, axis = 1)

pivot_speed_stands = begin_dataset.value_counts('speed').reset_index().rename(columns = {'count': 'Количество'})
pivot_speed_stands

pie_diagram_speed_stands = px.pie(pivot_speed_stands, names='speed', values='Количество', color_discrete_sequence=['#4aeb14', '#b514eb'], width=500, hole=0.5)
pie_diagram_speed_stands.update_layout(title = {'text': 'Распределение стендов по скорости', 'x': 0.5}, showlegend = False)
pie_diagram_speed_stands.show()

Да, все так у нас **в основном действительно быстрые стенды**

### 2.3.3 -- Дальность действия

Теперь проаналлизируем дальность действия стендов, то есть как далеко они могут атаковать/действовать

In [298]:
display(begin_dataset)

piv_range_table = begin_dataset.groupby('Дальность').size().reset_index().sort_values(0, ascending=False)

display(piv_range_table)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
...,...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд,Медленный стенд


,Дальность,0
2,3,37
4,5,37
3,4,25
0,1,21
1,2,21
7,undefined,8
5,infinite,4
8,unknown,2
6,none,1


Как видно, у нас в основном стенды дальнего действия, это может говорить нам о том, что их можно спокойно использовать на нескольких сотен метров и даже больше

У нас здесь есть 4 категории, давайте посмотрим на них поподробнее и узнаем, что это за стенды такие

In [299]:
unknown_range_stnads = begin_dataset[
    (begin_dataset['Дальность'] == 'undefined') | (begin_dataset['Дальность'] == 'infinite') | (begin_dataset['Дальность'] == 'unknown') | (begin_dataset['Дальность'] == 'none')
]

display(unknown_range_stnads)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed
33,Achtung Baby,1,1,undefined,5,1,5,Part 4: Diamond is Unbreakable,Слабый стенд,Медленный стенд
35,Atom Heart Father,1,1,undefined,5,1,1,Part 4: Diamond is Unbreakable,Слабый стенд,Медленный стенд
42,Earth Wind and Fire,3,3,undefined,5,3,3,Part 4: Diamond is Unbreakable,Сильный стенд,Быстрый стенд
53,Super Fly,1,1,none,5,1,1,Part 4: Diamond is Unbreakable,Слабый стенд,Медленный стенд
60,Stray Cat,4,1,undefined,5,1,3,Part 4: Diamond is Unbreakable,Сильный стенд,Медленный стенд
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд
83,Notorious B.I.G,5,infinite,infinite,infinite,1,5,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд
90,Whitesnake,5,2,unknown,5,unknown,unknown,Part 6: Stone Ocean,Сильный стенд,Медленный стенд
99,Bohemian Rhapsody,1,3,infinite,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Быстрый стенд
100,Burning Down the House,2,2,undefined,undefined,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Медленный стенд


У нас здесь есть несколько стендов, у которых дальность -- бесконечность, посмотрим на них внимательно

In [300]:
infinite_range_stnads = unknown_range_stnads[
    unknown_range_stnads['Дальность'] == 'infinite'
]

display(infinite_range_stnads)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд
83,Notorious B.I.G,5,infinite,infinite,infinite,1,5,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд
99,Bohemian Rhapsody,1,3,infinite,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Быстрый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд,Медленный стенд


Итак, 3 стенда отсюда мы уже знаем, а вот **Bohemian Rhapsody** это что-то новое, ~~я не верю, что за 11 последних эпизодов будет что-то еще))~~ давайте посмотрим на этот стенд...

Ага, интересно, этот стенд, как я понял, переносит вымышленных персонажей в реальный мир, то есть в нашем мире появляется человек-паук, Алладин, Красная шапочка и серый волк и так далее (подробнее можете почитать [здесь](https://jojo.fandom.com/ru/wiki/Bohemian_Rhapsody))

Тут написано:

> Убегать или прятаться от персонажей бесполезно, так как объекты просто разворачиваются как бумага, позволяя персонажам проходить сквозь них или даже телепортироваться

Думаю, что можно ставить 5 смело, так как стенд действительно работает на далеком расстоянии

Посмотрим на остальные стенды. Здесь уже интересно -- из главы *Diamond is Unbreakable* все стенды, кроме Super Fly я бы поставил дальность действия 5, честно. Потому что в аниме показано, что у них очень большой радиус действия, например у того же **Atom Heart Father**, он спокойно и незаметно перемещается по городу Морио

Исключения -- **Super Fly** и **Achtung Baby**, первое это прсото вышка, которая стоит на месте и радиус действия -- только ее местонахождение. А второй стенд -- это невидимый 1-2 летний ребенок, который еще не умеет разговаривать. Так что я бы дальность действия дал только 1, думаю, справедливо

Смотря на другие стенды, я бы скорее их бы распределил также, как силу и скорость, так как не знаю точно, на что они спопсобны. Думаю, будет честно и я никого не обижу

In [301]:
# Заменим определенные стенды с радиусом действия на 1 

diamond_is_unbreakeble_unknown_range_stand_one = begin_dataset[
    begin_dataset['Story'].str.startswith('Part 4') & ((begin_dataset['Stand'] == 'Achtung Baby') | (begin_dataset['Stand'] == 'Super Fly'))
]
diamond_is_unbreakeble_unknown_range_stand_one['Дальность'] = 1
begin_dataset.loc[diamond_is_unbreakeble_unknown_range_stand_one.index, 'Дальность'] = diamond_is_unbreakeble_unknown_range_stand_one['Дальность']

#display(begin_dataset.query('Stand == "Achtung Baby"'))

# Заменим определенные стенды с радиусом действия на 5

diamond_is_unbreakeble_unknown_range_stand_five = begin_dataset[
    begin_dataset['Story'].str.startswith('Part 4') & (begin_dataset['Дальность'] == 'undefined') & ((begin_dataset['Stand'] != 'Achtung Baby') | (begin_dataset['Stand'] != 'Super Fly'))
]
diamond_is_unbreakeble_unknown_range_stand_five['Дальность'] = 5
begin_dataset.loc[diamond_is_unbreakeble_unknown_range_stand_five.index, 'Дальность'] = diamond_is_unbreakeble_unknown_range_stand_five['Дальность']

#display(begin_dataset.query('Stand == "Stray Cat"'))


C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\194860243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\194860243.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Так, отлично, некоторые стенды по смыслу мы заполнили, а теперь надо еще дополнить значения у других стендов, заполним аналогично силе и скорости

In [302]:
know_range_stands = begin_dataset[
    (begin_dataset['Дальность'] != 'undefined') & (begin_dataset['Дальность'] != 'unknown') & (begin_dataset['Дальность'] != 'infinite') & (begin_dataset['Дальность'] != 'none')
]

pivot_range_stands = know_range_stands.value_counts('Дальность').reset_index()

display(know_range_stands)

histogram_range_stands = px.histogram(know_range_stands, x = 'Дальность', nbins = 8, color_discrete_sequence=['#efe05e'])

histogram_range_stands.update_layout(
    title = {'text': 'Распределение стендов по дальности действия', 'x': 0.5},
    xaxis_title = 'Дальность',
    yaxis_title = 'Количество')

histogram_range_stands.show()

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд
...,...,...,...,...,...,...,...,...,...,...
150,Dolly Dagger,5,5,3,5,4,3,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд


Заполним недостающие значения с помощью имеющихся данных, с определенной вероятностью

In [303]:
sum_range = pivot_range_stands['count'].sum()
pivot_range_stands['probability'] = pivot_range_stands['count'] / sum_range
pivot_range_stands

,Дальность,count,probability
0,5,40,0.273973
1,3,37,0.253425
2,4,25,0.171233
3,1,23,0.157534
4,2,21,0.143836


In [304]:
unknown_range_stands = begin_dataset[
    (begin_dataset['Дальность'] == 'undefined') | (begin_dataset['Дальность'] == 'unknown') | (begin_dataset['Дальность'] == 'none')
]

def replace_range_stands(row):
    row['Дальность'] = np.random.choice(pivot_range_stands['Дальность'], p = pivot_range_stands['probability'])
    return row['Дальность']

unknown_range_stands['Дальность'] = unknown_range_stands.apply(replace_range_stands, axis = 1)

begin_dataset.loc[unknown_range_stands.index, 'Дальность'] = unknown_range_stands['Дальность']

# display(begin_dataset.query('Stand == "Whitesnake"'))



C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\1035680595.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Теперь посмотрим на общее распределение стендов по дальности при помощи круговой диаграммы, поступим аналогично, как и для силы и скорости: 3 и больше -- стенды дальнего действия, меньше -- близкого

In [305]:
def assign_range_stands(row):
    if row['Дальность'] == 'infinite' or row['Дальность'] >= 3:
        row['range'] = 'Стенд дальнего действия'
    else:
        row['range'] = 'Стенд ближнего боя'
    return row['range']

begin_dataset['range'] = begin_dataset.apply(assign_range_stands, axis = 1)


pivot_table_range = begin_dataset.value_counts('range').reset_index().rename(columns={'count': 'Количество'})

display(pivot_table_range)

pie_chart_range = px.pie(pivot_table_range, names='range', values='Количество', width = 500, hole = 0.5, color_discrete_sequence=['#efe05e', '#5e6cef'])

pie_chart_range.update_layout(
    title = {'text': 'Распределение стендов по дальности действия', 'x': 0.5},
    showlegend = False)

pie_chart_range.show()


,range,Количество
0,Стенд дальнего действия,111
1,Стенд ближнего боя,45


Как видно, у нас **большинство стендов -- дальнего действия**

### 2.3.4 -- Выносливость

Теперь проанализируем выносливость у стендов

In [306]:
piv_stamina = begin_dataset.value_counts('Выносливость')

display(piv_stamina)

Выносливость
5            84
4            35
3            21
1             6
2             5
infinite      2
unknown       2
undefined     1
Name: count, dtype: int64

Ого, да у нас тут все стенды обладают высокой выносливаостью, хорошо проявляют себя в бою и довольно стойкие. В любом случае посмотрим на неизвестные стенды и попробуем сообразить, на что нам их заменить

In [307]:
unknown_stamina_stands = begin_dataset[
    (begin_dataset['Выносливость'] == 'unknown') | (begin_dataset['Выносливость'] == 'infinite') | (begin_dataset['Выносливость'] == 'undefined')
]

display(unknown_stamina_stands)

unknown_stamina_stands = begin_dataset[
    (begin_dataset['Выносливость'] != 'unknown') | (begin_dataset['Выносливость'] != 'infinite') | (begin_dataset['Выносливость'] != 'undefined')
]
histogram_stamina_stands = px.histogram(unknown_stamina_stands, x = 'Выносливость', nbins = 8, color_discrete_sequence=['#ff00c3'])

histogram_stamina_stands.update_layout(
    title = {'text': 'Распределение стендов по выносливости', 'x': 0.5},
    xaxis_title = 'Выносливость',
    yaxis_title = 'Количество')

histogram_stamina_stands.show()

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия
83,Notorious B.I.G,5,infinite,infinite,infinite,1,5,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия
91,C-Moon,4,4,4,unknown,unknown,unknown,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия
100,Burning Down the House,2,2,3,undefined,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Медленный стенд,Стенд дальнего действия
104,"Green, Green Grass of Home",5,4,4,unknown,unknown,unknown,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия


Так, ну с GER и Notorious B.I.G понятно, **C-Moon** -- это вроде реквием Пуччи.... наверное...., так что поставим, наверное, 5 за выносливость :)

**Burning Down the House** -- стенд мальчика, Эмпорио, это дом-призрак, он действует постоянно и, если исходить из того, что это такой хороший дом-укрытие, то, думаю, стоит поставить ему выносливость 5, так как дом всегда доступен Эмпорио и тем, кто вместе с ним (подробнее почитать [здесь](https://jojo.fandom.com/ru/wiki/Burning_Down_the_House))

<img src="https://static.wikia.nocookie.net/jojo/images/0/00/BDtH_Infobox_Manga.png/revision/latest?cb=20211214042954&path-prefix=ru" alt="burning_down_the_house" width="200"/>

**Green, Green Grass of Home** судя по описанию защищает младенца, уменьшая любого человека, который приближается к нему, он автоматического типа, так что думаю, что и ему можно поставить 5 по выносливости (подробнее можно почитать [здесь](https://jojo.fandom.com/ru/wiki/Green,_Green_Grass_of_Home))


<img src="
https://static.wikia.nocookie.net/jojo/images/a/a9/G3oH_Infobox_Manga.png/revision/latest?cb=20220823173851&path-prefix=ru
" alt="burning_down_the_house" width="200"/>


In [308]:
unknown_stamina_stands = begin_dataset[
    (begin_dataset['Выносливость'] == 'unknown') | (begin_dataset['Выносливость'] == 'undefined')
]

unknown_stamina_stands['Выносливость'] = 5


begin_dataset.loc[unknown_stamina_stands.index, 'Выносливость'] = unknown_stamina_stands['Выносливость']

#display(begin_dataset.query('Stand == "C-Moon"'))

C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\4279200455.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Так же, как и с прошлыми характеристиками сделаем так, что выносливость, значение которой >= 3 -- выносливые, стойкие стенды, меньше -- стенд доволььно уязвимый 

In [309]:
def assign_stamina_stands(row):
    if row['Выносливость'] == 'infinite' or row['Выносливость'] >= 3:
        row['stamina'] = 'Выносливый, стойкий стенд'
    else:
        row['stamina'] = 'Стенд довольно уязвимый'
    return row['stamina']

begin_dataset['stamina'] = begin_dataset.apply(assign_stamina_stands, axis = 1)

pivot_table_stamina = begin_dataset.value_counts('stamina').reset_index().rename(columns={'count': 'Количество'})

display(pivot_table_stamina)

stamina_pie_diagram = px.pie(pivot_table_stamina, values='Количество', names = 'stamina', hole = 0.5, width = 500, color_discrete_sequence=['#ff00c3', '#00ff3c'])
stamina_pie_diagram.update_layout(title = {'text': 'Распределение стендов по выносливости/стойкости', 'x': 0.5}, showlegend = False)
stamina_pie_diagram.show()

,stamina,Количество
0,"Выносливый, стойкий стенд",145
1,Стенд довольно уязвимый,11


Как мы видим, **почти все стенды обладают высокой выносливостью/стойкостью в бою**

### 2.3.5 -- Точность

Теперь посмотрим, как у нас дела с точностью у стендо. Высокая точность указывает на то, что стенд может точно наносить удары, контролировать свои действия и выполнять сложные манипуляции с высокой степенью успеха. Этот параметр важен для стендов, которые требуют высокой степени контроля и координации

In [310]:
display(begin_dataset.value_counts('Точность'))

Точность
1              55
3              40
2              25
5              12
4              11
undefined       7
unknown         4
infinite        1
situational     1
Name: count, dtype: int64

Как видно, у нас много стендов с низкой точностью, а также из интересного: у нас есть один стенд, у которого точность _ситуативная_

Посмотрим поподробнее на необычные стенды

In [311]:
unknown_accuracy_stands = begin_dataset[
    (begin_dataset['Точность'] == 'undefined') | (begin_dataset['Точность'] == 'unknown') | (begin_dataset['Точность'] == 'infinitive') |
    (begin_dataset['Точность'] == 'situational')
]

display(unknown_accuracy_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina
62,Baby Face,5,4,5,5,situational,situational,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
79,King Crimson,5,5,1,1,unknown,unknown,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый
90,Whitesnake,5,2,2,5,unknown,unknown,Part 6: Stone Ocean,Сильный стенд,Медленный стенд,Стенд ближнего боя,"Выносливый, стойкий стенд"
91,C-Moon,4,4,4,5,unknown,unknown,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
92,Jail House Lock,1,3,4,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
95,Under World,5,3,5,3,undefined,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
96,Sky High,4,3,4,3,undefined,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
99,Bohemian Rhapsody,1,3,infinite,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
100,Burning Down the House,2,2,3,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
102,Dragon's Dream,3,4,3,5,undefined,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"


**Baby face** -- интересный стенд, являющийся ребенком, способности которого действительно ситуативные и зависят от матери. Подробнее можно почитать [здесь](jojo.fandom.com/ru/wiki/Baby_Face)

<img src="
https://static.wikia.nocookie.net/jojo/images/c/ca/Baby_Face_Junior_Infobox_Anime.png/revision/latest?cb=20231111155315&path-prefix=ru" alt="burning_down_the_house" width="200"/>

Насчет **King Crimson** я уверен, можно точности ставить 5, так как стенд и правда очень сильный. А остальные стенды можно заполнить привычным нам способом

In [312]:
begin_dataset.loc[begin_dataset[begin_dataset['Stand'].str.startswith('King')].index, 'Точность'] = 5

#display(begin_dataset.query('Stand == "King Crimson"'))

known_accuracy_stands = begin_dataset[
    (begin_dataset['Точность'] != 'undefined') & (begin_dataset['Точность'] != 'unknown') & (begin_dataset['Точность'] != 'infinite')
    & (begin_dataset['Точность']!='situational')
]

piv_accuracy_table = known_accuracy_stands.value_counts('Точность').reset_index()


accuracy_hist = px.histogram(known_accuracy_stands, x='Точность', color_discrete_sequence=['#03a5fc'])
accuracy_hist.update_layout(title = {'text': 'Распределение стендов по точности', 'x': 0.5}, showlegend = False)
accuracy_hist.show()

sum_accuracy = piv_accuracy_table['count'].sum()

piv_accuracy_table['probability'] = piv_accuracy_table['count'] / sum_accuracy


display(piv_accuracy_table)


,Точность,count,probability
0,1,55,0.381944
1,3,40,0.277778
2,2,25,0.173611
3,5,13,0.090278
4,4,11,0.076389


In [313]:
unknown_accuracy_stands = begin_dataset[
    (begin_dataset['Точность'] == 'undefined') | (begin_dataset['Точность'] == 'unknown') |
    (begin_dataset['Точность'] == 'situational')
]


def assign_accuracy_stands(row):
    row['Точность'] = np.random.choice(piv_accuracy_table['Точность'], p = piv_accuracy_table['probability'])
    return row['Точность']

unknown_accuracy_stands['Точность'] = unknown_accuracy_stands.apply(assign_accuracy_stands, axis = 1)

begin_dataset.loc[unknown_accuracy_stands.index, 'Точность'] = unknown_accuracy_stands['Точность']

display(begin_dataset)

#display(begin_dataset.query('Stand == "Whitesnake"'))



C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\3061866888.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд"
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд"
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд"
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд"
...,...,...,...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд"


Точные и неточные стенды определяем таким же образом -- **точность >= 3 -- точный стенд, иначе -- неточный**

In [314]:
def assign_categories_accuracy(row):
    if row['Точность'] == 'infinite' or row['Точность'] >= 3:
        row['accuracy'] = 'Точный стенд'
    else:
        row['accuracy'] = 'Неточный стенд'
    return row['accuracy']

begin_dataset['accuracy'] = begin_dataset.apply(assign_categories_accuracy, axis = 1)

pivot_table_accuracy = begin_dataset.value_counts('accuracy').reset_index().rename(columns={'count': 'Количество'})

display(pivot_table_accuracy)

accuracy_pie_diagram = px.pie(pivot_table_accuracy, values='Количество', names = 'accuracy', hole = 0.5, width = 500, color_discrete_sequence=['#fc5a03', '#03a5fc'])

accuracy_pie_diagram.update_layout(title = {'text': 'Распределение стендов по точности/неточности', 'x': 0.5}, showlegend = False)

accuracy_pie_diagram.show()

,accuracy,Количество
0,Неточный стенд,86
1,Точный стенд,70


Как видно, у нас **больше половины стендов -- неточные**

* Стенды с низкой точностью могут быть менее предсказуемыми и стабильными в своих действиях. Это может означать, что их атаки или способности могут не всегда попадать в цель или действовать так, как задумано
* Низкая точность может указывать на то, что пользователь стенда имеет ограниченный контроль над его действиями. Это может затруднить выполнение сложных маневров или точных атак
* Но это также может означать, что некоторые стенды с низкой точностью могут компенсировать это за счет своих широкомасштабных или разрушительных способностей. Например, стенды, которые создают большие взрывы или разрушения, могут не нуждаться в высокой точности, так как их основная цель — нанести максимальный урон в широкой области, например, **Killer Queen**

### 2.3.6 -- Потенциал

Остался последний параметр -- потенциал, после чего можно будет перейти к более подробному раскрытию стендов и глав аниме/манги, а пока надо поработать с грязными данными, увы и ах :(

In [315]:
display(begin_dataset.value_counts('Потенциал'))

Потенциал
3              40
1              39
5              25
2              20
4              16
undefined       9
unknown         5
infinite        1
situational     1
Name: count, dtype: int64

Можно с уверенностью сказать, что ~у половины стендов высокий потенциал развития, сейчас посмотрим на стенды, которые находятся под таинстенными _undefined_, _unknown_ и прочие знакомые нам значения

In [316]:
unknown_potential_stands = begin_dataset[
    (begin_dataset['Потенциал'] == 'undefined') | (begin_dataset['Потенциал'] == 'unknown') |
    (begin_dataset['Потенциал'] == 'infinite') | (begin_dataset['Потенциал'] == 'situational')
]

display(unknown_potential_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy
62,Baby Face,5,4,5,5,3,situational,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд
79,King Crimson,5,5,1,1,5,unknown,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый,Точный стенд
85,Rolling Stones,5,4,5,5,1,undefined,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
90,Whitesnake,5,2,2,5,1,unknown,Part 6: Stone Ocean,Сильный стенд,Медленный стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд
91,C-Moon,4,4,4,5,1,unknown,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
92,Jail House Lock,1,3,4,5,2,undefined,Part 6: Stone Ocean,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
95,Under World,5,3,5,3,2,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
96,Sky High,4,3,4,3,4,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд
97,Star Platinum,5,5,3,5,5,undefined,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд


Опять у нас появился **Baby Face**, потенциал у него ситуативный, но, рассмотрим лучший случай и когда нащ "зародыш" появляется от лучшей матери ~~чзх~~, так что в потенциал поставим ему 5 баллов

**King Crimson**, как будто, достиг совего потенциала, и, думаю, что где-то заслуженная 4 ему обеспечена

О, а вот и **Star Platinum** появился, наконец-таки, вообще, я слышал, что его в 6 части просто слили, но в принципе и понятно: там Джотаро в каком эпизоде умер? В 8? Неважно, я его запомнил со времен 3 сезона и могу сказать наверняка: если бы Джотаро бы его развивал, то я уверен, что он бы останавливал время и на большее количество минут. Потенциал одназночно был, поэтому ставим 5 баллов

А, тем более у нас тут несколько **Star Platinum**, поэтому 100% ставим максимум -- 5

Насчет остальных неуверен, поэтому пойдем обычным путем

In [317]:
stands_with_potential_5 = begin_dataset[
    (begin_dataset['Stand'].str.startswith('Baby')) | (begin_dataset['Stand'].str.startswith('Star'))
]

stands_with_potential_4 = begin_dataset[
    (begin_dataset['Stand'].str.startswith('King'))
]

stands_with_potential_5['Потенциал'] = 5
stands_with_potential_4['Потенциал'] = 4

begin_dataset.loc[stands_with_potential_4.index, 'Потенциал'] = stands_with_potential_4['Потенциал']
begin_dataset.loc[stands_with_potential_5.index, 'Потенциал'] = stands_with_potential_5['Потенциал']


C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\33348233.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\33348233.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [318]:
known_potential_stands = begin_dataset[
    (begin_dataset['Потенциал'] != 'undefined') & (begin_dataset['Потенциал'] != 'unknown') & (begin_dataset['Потенциал'] != 'infinite')
]

pivot_table_potential = known_potential_stands.value_counts('Потенциал').reset_index().rename(columns={'count': 'Количество'})


potential_histogram = px.histogram(pivot_table_potential, x = 'Потенциал', y = 'Количество', nbins = 8, color_discrete_sequence=['#00ff17'])

potential_histogram.update_layout(title = {'text': 'Распределение стендов по потенциалу развития', 'x': 0.5})

potential_histogram.show()

sum_potential = pivot_table_potential['Количество'].sum()

pivot_table_potential['probability'] = pivot_table_potential['Количество'] / sum_potential

display(pivot_table_potential)

,Потенциал,Количество,probability
0,1,39,0.272727
1,3,38,0.265734
2,5,29,0.202797
3,2,20,0.139860
4,4,17,0.118881


In [319]:
unknown_potential_stands = begin_dataset[
    (begin_dataset['Потенциал'] == 'undefined') | (begin_dataset['Потенциал'] == 'unknown')
]

def replace_empty_potential(row):
    row['Потенциал'] = np.random.choice(pivot_table_potential['Потенциал'], p = pivot_table_potential['probability'])
    return row['Потенциал']

unknown_potential_stands['Потенциал'] = unknown_potential_stands.apply(replace_empty_potential, axis = 1)

begin_dataset.loc[unknown_potential_stands.index, 'Потенциал'] = unknown_potential_stands['Потенциал']

display(begin_dataset)

C:\Users\Smart\AppData\Local\Temp\ipykernel_13008\1976422178.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Точный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
152,Purple Haze Distortion,5,4,1,1,3,5,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый,Точный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд


Строим диаграмму таким же образом: **стенды с высоким потенциалом развития -- потенциал >= 3, иначе -- потенциал низкий**

In [320]:
def assign_potential_stands(row):
    if row['Потенциал'] == 'infinite' or row['Потенциал'] >= 3:
        row['potential'] = 'Высокий потенциал'
    else:
        row['potential'] = 'Низкий потенциал'
    return row['potential']

begin_dataset['potential'] = begin_dataset.apply(assign_potential_stands, axis = 1)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Точный стенд,Низкий потенциал
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал
152,Purple Haze Distortion,5,4,1,1,3,5,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый,Точный стенд,Высокий потенциал
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Низкий потенциал


In [321]:
pivot_potential_table = begin_dataset.value_counts('potential').reset_index().rename(columns = {'count': 'Количество'})

display(pivot_potential_table)

potential_pie_diagram = px.pie(pivot_potential_table, names = 'potential', values = 'Количество', hole = 0.5, width = 500, color_discrete_sequence=['#00ff17', '#ff00e8'])
potential_pie_diagram.update_layout(title = {'text': 'Распределение стендов с потенциалом', 'x': 0.5}, showlegend = False)

potential_pie_diagram.show()

,potential,Количество
0,Высокий потенциал,93
1,Низкий потенциал,63


Как видно, **стендов с высоким потенциалом все же больше**

Все, с утомительной обработкой грязных данных закончили, **теперь пора рассмотреть самый сок -- сами стенды в каждом сезоне/главе, выявить самые сильные, слабые, и потом еще срвнить стенды потом в каждом сезоне и понять, в каком же сезоне среднестатистический стенд наиболее сильный, а также предлагаю разбивать стенды в каждом сезоне на группы/фракции (дальше наглядно покажу все)**


## 2.4 -- Общая мера измерения

Перед тем, как рассматривать отдельные главы нам надо определиться, пожалуй, с одним из главных вопросов в этом датасете -- **как мы будем оценивать стенды?**

Я предлагаю присваивать каждому стенду общий балл от 1 до 6 (от самого слабого (все параметры _E_) к самому сильному (все параметры _infinite_))

Соответсвенно, нам надо как-то присвоить коэффициенты для характеристик, чтобы они (коэф.) в сумме давали нам 1. Это одна из фундаментальных частей, так как от этого будет зависеть -- попадет очередный стенд в топ или нет

**Поэтому перед тем как предоставить коэффициенты я сделаю важную оговорочку:**

**ЭТО ВСЕ МОЕ ЛИЧНОЕ МНЕНИЕ, КОТОРОЕ ОПИРАЕТСЯ НА МОЕ ЛИЧНОЕ ВОСПРИЯТИЕ СТЕНДОВ И ЗАСЧЕТ ГРАФИКОВ РАСПРЕДЕЛЕНИЙ, КОТОРЫЕ МЫ С ВАМИ ПОСТРОИЛИ**

In [322]:
display(begin_dataset.sample(3))

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential
26,Strength,4,2,2,5,1,1,Part 3: Stardust Crusaders,Сильный стенд,Медленный стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал
79,King Crimson,5,5,1,1,5,4,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый,Точный стенд,Высокий потенциал
117,Ball Breaker,5,5,2,4,3,4,Part 7: Steel Ball Run,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал


In [323]:
proof_to_accuracy = begin_dataset.query('Stand == "Gold Experience"')
proof_to_accuracy


,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential
63,Gold Experience,3,5,3,2,3,5,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,Стенд довольно уязвимый,Точный стенд,Высокий потенциал


Итак, мое распределение коэффициентов к каждому навыку во время боя:


1) Я считаю, что в бою в первую очередь важна **выносливость**, так как она позволяет продержаться стенду максимально долго <br> Давайте присвоим выносливости коэффициент, равный **0.25** 
   
2) Также второй важный параметр, который тоже очень важный во время схватки это **сила**, так как чем больше ее у стенда, тем более достойный отпор даст стенд, а соотвественно и шансы на выживание повышается <br> Давайте присвоим силе коэффициент, равный **0.2**

3) Третий важный параметр это **потенциал развития**, так как мое субъективное мнение -- именно благодаря высокому потенциалу развития и быстрой адаптации к бою, в 3 сезоне тот же Star Platinum смог побеждать толпы врагов. К тому же потенциал развития стенда определяет его способность к росту и улучшению. Высокий потенциал развития позволяет стенду становиться сильнее и эффективнее со временем, это важная характеристика для долгосрочных стратегий и развития персонажа <br> Поэтому давайте присвоим потенциалу коэффициент, равный **0.2**

4) Четвертая важная характеристика это **скорость**, так как если стенд будет слишком медленный, то он не сможет увернуться даже от обычного удара и может проиграть бой. <br> Пускай коэффициент у этого параметра будет **0.17** 


5) Следующая на очереди характеристика у нас это **дальность**, я думаю, что если у стенда будет хорошая дальность атаки, выносливость и сила, то это поможет стенду быть эффективным на любых расстояних и, как вариант, быстрее расправляться с противником, не привязываясь тоьлко к ближнему бою <br> Пускай коэффициент у дальности будет **0.13**

6) И последний, но не по значению параметр -- **точность**. Объясню. Возьмем, например, Джорно Джованну и его стенд _Gold Experience_ (не реквием), стенд поистине могущественный и сильный. Он способен наносить точные удары в ближнем бою, но когда дело касается образования жизни, то он здесь может быть и не совсем точным. Там свои шероховатости, конечно, но все же, ему низкая точность не помешала стать одним из сильных стендов <br> Поэтому этот параметр получает суровую, наверное, оценку в **0.05**

Таким образом, получаем следующую табличку, на которую мы будем опираться при оценивании стендов

In [324]:
done_coef_table = pd.DataFrame({
    'Параметры': ['Выносливость', 'Сила', 'Потенциал развития', 'Скорость', 'Дальность', 'Точность'],
    'Коэффициенты': [0.25, 0.2, 0.2, 0.17, 0.13, 0.05]
})

display(done_coef_table)


,Параметры,Коэффициенты
0,Выносливость,0.25
1,Сила,0.20
2,Потенциал развития,0.20
3,Скорость,0.17
4,Дальность,0.13
5,Точность,0.05


Теперь на основе данных коэффициентов рассчитаем общий балл каждого стенда от 1 до 6 (E - _infinite_)

Заменим параметры, у которых написано _infinite_ на 6

In [325]:
stats_columns = begin_dataset.columns[1:7]

begin_dataset[stats_columns] = begin_dataset[stats_columns].replace({'infinite': '6'})

for i in stats_columns:
    #display(begin_dataset.value_counts(i))
    begin_dataset[i] = begin_dataset[i].astype(float)

begin_dataset['Общий балл стенда'] = (begin_dataset['Сила'] * 0.2 + begin_dataset['Выносливость'] * 0.25 
+ begin_dataset['Потенциал'] * 0.2 + begin_dataset['Скорость'] * 0.17 + begin_dataset['Дальность'] * 0.13 
+ begin_dataset['Точность'] * 0.05)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential,Общий балл стенда
0,Anubis,4.0,4.0,1.0,5.0,1.0,3.0,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал,3.51
1,Atum,2.0,3.0,2.0,4.0,2.0,2.0,Part 3: Stardust Crusaders,Слабый стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал,2.67
2,Bastet,1.0,1.0,4.0,5.0,1.0,1.0,Part 3: Stardust Crusaders,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал,2.39
3,Cream,4.0,4.0,2.0,3.0,3.0,2.0,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Точный стенд,Низкий потенциал,3.04
4,Death Thirteen,3.0,3.0,1.0,4.0,2.0,4.0,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд ближнего боя,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал,3.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1.0,5.0,5.0,5.0,1.0,1.0,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал,3.20
152,Purple Haze Distortion,5.0,4.0,1.0,1.0,3.0,5.0,Light Novel: Purple Haze Feedback,Сильный стенд,Быстрый стенд,Стенд ближнего боя,Стенд довольно уязвимый,Точный стенд,Высокий потенциал,3.21
153,Rainy Day Dream Away,1.0,4.0,5.0,5.0,1.0,1.0,Light Novel: Purple Haze Feedback,Слабый стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Низкий потенциал,3.03
154,Remote Romance,1.0,2.0,6.0,4.0,3.0,1.0,Featured,Слабый стенд,Медленный стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Низкий потенциал,2.67


Посмотрим, как у нас распределились общие баллы

In [326]:
overall_df = begin_dataset.value_counts('Общий балл стенда').reset_index()

display(overall_df.sort_values('Общий балл стенда', ascending=False))

overall_hist = px.histogram(overall_df, x = 'Общий балл стенда', y='count', color_discrete_sequence=['#00baff'])

overall_hist.update_layout(
    title = {'text': 'Распределение общих баллов стендов', 'x': 0.5},
    xaxis_title_text='Общий балл стенда',
    yaxis_title_text='Количество стендов')
overall_hist.show()

,Общий балл стенда,count
127,6.00,1
43,5.35,1
16,4.90,2
2,4.74,3
19,4.73,2
...,...,...
0,2.00,6
96,1.91,1
97,1.90,1
104,1.75,1


Как видно, у нас в основном стенды имеют средний бал ~3.25, а это значит, что в основном у нас стенды средние и очень сильных стендов крайне мало, по крайней мере меньше, чем слабых :)

## 2.5 -- Топ 10 самых сильных стендов

Посмотрим на топ 10 самых сильных стендов

In [327]:
display(begin_dataset.sort_values('Общий балл стенда', ascending=False).head(10))

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential,Общий балл стенда
77,Gold Experience Requiem,6.0,6.0,6.0,6.0,6.0,6.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,6.00
83,Notorious B.I.G,5.0,6.0,6.0,6.0,1.0,5.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал,5.35
52,Red Hot Chili Pepper,5.0,5.0,5.0,5.0,3.0,5.0,Part 4: Diamond is Unbreakable,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.90
106,Kiss,5.0,5.0,5.0,5.0,3.0,5.0,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.90
97,Star Platinum,5.0,5.0,3.0,5.0,5.0,5.0,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.74
25,Star Platinum,5.0,5.0,3.0,5.0,5.0,5.0,Part 3: Stardust Crusaders,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.74
123,Dirty Deeds Done Dirt Cheap,5.0,5.0,3.0,5.0,5.0,5.0,Part 7: Steel Ball Run,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.74
62,Baby Face,5.0,4.0,5.0,5.0,3.0,5.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.73
73,Baby Face (GoGo),5.0,4.0,5.0,5.0,3.0,5.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.73
109,Made in Heaven,4.0,6.0,3.0,5.0,3.0,5.0,Part 6: Stone Ocean,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.61


Да, самый сильный оказался **GER**, оно и очевидно

Давайте теперь отобразим все эти 10 стендов на культовой радиальной диаграмме, чтобы было красиво :)

### **1 место -- Gold Experience Requiem**

<p align="center">
    <img src="https://i.pinimg.com/originals/6a/cc/c6/6accc6d646fb378c932fcd2b73e16307.png" alt="Gold Experience Requiem 2" width="300">
</p>

<p align="center">
    <img src="https://steamuserimages-a.akamaihd.net/ugc/5108802431775556322/46F42EC084464AA428E7703388BA664F0A560A9C/?imw=512&amp;imh=287&amp;ima=fit&amp;impolicy=Letterbox&amp;imcolor=%23000000&amp;letterbox=true" alt="Gold Experience Requiem 3" width="300">
</p>


In [328]:
GER_stats = begin_dataset[
    begin_dataset['Stand'].str.startswith('Gold Experience Requiem')
]
display(GER_stats)

GER_list = GER_stats[stats_columns].values.tolist()[0]



GER_radar_chart = px.line_polar(r = GER_list, 
                                theta = ['Сила', 'Скорость', 'Дальность', 'Выносливость', 'Точность', 'Потенциал'],
                                line_close=True,
                                color_discrete_sequence=['#f0d70f'],
                                width = 500, 
                                markers=True,
                                )

GER_radar_chart.update_layout(title = {'text': 'Характеристики стенда Gold Experience Requiem', 'x':  0.5, 'font_color': 'white'}, 
                              paper_bgcolor = '#735404',
                              polar = {'bgcolor': '#a16605', 
                                       'angularaxis': {'color': 'white'},
                                       'radialaxis': {'color': 'white'}},
                              )

GER_radar_chart.update_traces(fill='toself')
GER_radar_chart.show()

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential,Общий балл стенда
77,Gold Experience Requiem,6.0,6.0,6.0,6.0,6.0,6.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,6.0


### **2 место -- Notorious B.I.G**


<p align="center">
    <img src="https://i.ytimg.com/vi/qvk0zrIqm4w/maxresdefault.jpg" alt="Да... Страшный стенд" height = "200", width="400">
</p>

In [331]:
Notorius_df = begin_dataset[
    begin_dataset['Stand'] == 'Notorious B.I.G'
]
display(Notorius_df)

Notorius_val = Notorius_df[stats_columns].values.tolist()[0]


Notorius_chart = px.line_polar(r = Notorius_val,
                               theta=stats_columns,
                               line_close=True,
                               markers=True,
                               color_discrete_sequence=['#af5092'],
                               width = 500)
Notorius_chart.update_layout(title = {'text': 'Характеристики стенда Notorious B.I.G', 'x': 0.5, 'font_color': 'white'},
                             paper_bgcolor = '#d02f39',
                             polar = {'bgcolor': '#f5d2d5',
                                       'angularaxis': {'color': 'white', 'linecolor': 'black', 'gridcolor': '#572c76'},
                                       'radialaxis': {'color': 'black', 'linecolor': 'black', 'gridcolor': '#572c76'}}
                             )
Notorius_chart.update_traces(fill = 'toself')
Notorius_chart.show()

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential,Общий балл стенда
83,Notorious B.I.G,5.0,6.0,6.0,6.0,1.0,5.0,Part 5: Vento Aureo,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Неточный стенд,Высокий потенциал,5.35


### **3 место -- Red Hot Chili Pepper**

<p align="center">
    <img src="https://avatars.mds.yandex.net/i?id=d391b2a9949406c5444703a3627584ef_l-4821100-images-thumbs&n=13" alt="Да... Страшный стенд" height = "200", width="400">
</p>

In [345]:
red_hot_pepper_df = begin_dataset[
    begin_dataset['Stand'] == 'Red Hot Chili Pepper'
]

display(red_hot_pepper_df)

red_hot_pepper_val = red_hot_pepper_df[stats_columns].values.tolist()[0]

red_hot_polar = px.line_polar(r = red_hot_pepper_val,
                              theta=stats_columns, 
                              markers=True,
                              line_close=True,
                              width = 500, 
                              color_discrete_sequence=['#f5e60a'],
                              )

red_hot_polar.update_layout(title = {'text': 'Характеристики стенда Red Hot Chili Pepper', 'x': 0.5, 'font_color': 'white'},
                            paper_bgcolor = '#e4331b',
                            polar = {'bgcolor': '#e4331b',
                                       'angularaxis': {'color': 'white', 'linecolor': 'white', 'gridcolor': 'white'},
                                       'radialaxis': {'color': 'white', 'linecolor': 'white', 'gridcolor': 'white'}}
                             )
red_hot_polar.update_traces(fill = 'toself')
red_hot_polar.show()

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong,speed,range,stamina,accuracy,potential,Общий балл стенда
52,Red Hot Chili Pepper,5.0,5.0,5.0,5.0,3.0,5.0,Part 4: Diamond is Unbreakable,Сильный стенд,Быстрый стенд,Стенд дальнего действия,"Выносливый, стойкий стенд",Точный стенд,Высокий потенциал,4.9


# Выводы


Большинство стендов обладают огромной силой, очень стойкие, быстрые, обладающие высоким потенциалом и к тому же еще и дальнего действия, но больше половины стендов имеют низкую точность

Самый сильный стенд всех времен и народов -- GER (Gold Experience Requiem)